In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import time
import re
from urllib.request import urlopen
import json
from pandas.io.json import json_normalize
import pandas as pd, numpy as np

In [24]:
username='natgeo'
browser = webdriver.Chrome('/Users/matt/Downloads/chromedriver')
browser.get('https://www.instagram.com/'+username+'/')

In [70]:
links=[]

def scrape(start, end):
    print('scrolling: {}-{}'.format(start, end))
    Pagelength = browser.execute_script("window.scrollTo({}, {});".format(start, end))
    source = browser.page_source
    data=bs(source, 'html.parser')
    body = data.find('body')
    script = body.find('span')
    for link in script.findAll('a'):
         if re.match("/p", link.get('href')):
            links.append('https://www.instagram.com'+link.get('href'))
    
    time.sleep(5)
    if (end < 216000):
        scrape(start + 3000, end + 3000)

In [71]:
scrape(0, 3000)

scrolling: 0-3000
scrolling: 3000-6000
scrolling: 6000-9000
scrolling: 9000-12000
scrolling: 12000-15000
scrolling: 15000-18000
scrolling: 18000-21000
scrolling: 21000-24000
scrolling: 24000-27000
scrolling: 27000-30000
scrolling: 30000-33000
scrolling: 33000-36000
scrolling: 36000-39000
scrolling: 39000-42000
scrolling: 42000-45000
scrolling: 45000-48000
scrolling: 48000-51000
scrolling: 51000-54000
scrolling: 54000-57000
scrolling: 57000-60000
scrolling: 60000-63000
scrolling: 63000-66000
scrolling: 66000-69000
scrolling: 69000-72000
scrolling: 72000-75000
scrolling: 75000-78000
scrolling: 78000-81000
scrolling: 81000-84000
scrolling: 84000-87000
scrolling: 87000-90000
scrolling: 90000-93000
scrolling: 93000-96000
scrolling: 96000-99000
scrolling: 99000-102000
scrolling: 102000-105000
scrolling: 105000-108000
scrolling: 108000-111000
scrolling: 111000-114000
scrolling: 114000-117000
scrolling: 117000-120000
scrolling: 120000-123000
scrolling: 123000-126000
scrolling: 126000-129000
sc

In [72]:
len(links)

1728

In [73]:
result=pd.DataFrame()
for i in range(len(links)):
    try:
        page = urlopen(links[i]).read()
        data=bs(page, 'html.parser')
        body = data.find('body')
        script = body.find('script')
        raw = script.text.strip().replace('window._sharedData =', '').replace(';', '')
        json_data=json.loads(raw)
        posts =json_data['entry_data']['PostPage'][0]['graphql']
        posts= json.dumps(posts)
        posts = json.loads(posts)
        x = pd.DataFrame.from_dict(json_normalize(posts), orient='columns') 
        x.columns =  x.columns.str.replace("shortcode_media.", "")
        result=result.append(x)       
    except:
        np.nan

result.index = range(len(result.index))

In [74]:
len(result)

1727

In [75]:
properties = ['__typename', 'display_url', 'edge_media_to_caption.edges', 'edge_media_preview_like.count', 'edge_media_preview_comment.count']
columns = ['type', 'media_url', 'caption', 'likes', 'comments']

output = result.copy()
output = output[properties]
output.columns = columns
output.caption = output.caption.apply(lambda s: str(s)
  .replace("[{'node': {'text': ", '')
  .replace("'", '')
  .replace("\"", ''))

output = output.drop_duplicates()
print('scraped {} posts'.format(len(output)))
output.head(10)

scraped 1144 posts


,type,media_url,caption,likes,comments
0,GraphSidecar,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by @erintrieb I In frigid temperatures j...,56008,201
1,GraphVideo,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,"With unprecedented access, THE NIGHTCRAWLERS f...",61898,771
2,GraphImage,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by @stevewinterphoto | One of the lucky ...,256591,1271
3,GraphImage,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by Charlie Hamilton James @chamiltonjame...,587042,1692
4,GraphImage,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by @babaktafreshi | A bird of freedom fl...,312169,3039
5,GraphSidecar,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photos by @timlaman | Wild baby orangutan upda...,460300,5257
6,GraphSidecar,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photos by @brentstirton | // Supported by Nat ...,272227,982
7,GraphVideo,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Video by David Chancellor @chancellordavid | I...,116490,539
8,GraphImage,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by Sara Hylton @sarahyltonphoto | Women ...,153079,394
9,GraphImage,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by Ian Teh @iantehphotography | Palm oil...,163279,618


In [77]:
images = output[output.type == 'GraphImage']
print('Images length: {}'.format(len(images)))
images.to_csv('output.csv', columns=columns, index=False)

Images length: 720
